In [1]:
import numpy as np

In [2]:
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

In [3]:
# importing the data
raw_data_np = np.genfromtxt('loan-data.csv', 
                            delimiter = ';', 
                            skip_header = 1, 
                            autostrip = True)
raw_data_np

In [4]:
# checking for incomplete data 
np.isnan(raw_data_np).sum()

In [5]:
# a filler for all the missing entries of the dataset
temporary_fill = np.nanmax(raw_data_np) + 1

# hold the means for every column
temporary_mean = np.nanmean(raw_data_np, axis = 0)

In [6]:
# to determine how many columns consist entirely of text values
temporary_mean

In [7]:
# to extract the minimum and maximum values for each numeric column
temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0),
                           temporary_mean,
                           np.nanmax(raw_data_np, axis = 0)])

In [8]:
temporary_stats

In [9]:
# column contains only text -> mean = nan -> np.isnan() = True -> True != 0 -> np.argwhere() returns the index
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_strings

In [10]:
# column contains numeric values
columns_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numeric

In [11]:
# re-importing the dataset as two separate arrays
# the string dataset
loan_data_strings = np.genfromtxt('loan-data.csv', 
                            delimiter = ';', 
                            skip_header = 1, 
                            autostrip = True, 
                            usecols = columns_strings, 
                            dtype = str)
loan_data_strings

In [12]:
# the numeric dataset
loan_data_numeric = np.genfromtxt('loan-data.csv', 
                            delimiter = ';', 
                            skip_header = 1, 
                            autostrip = True, 
                            usecols = columns_numeric, 
                            filling_values = temporary_fill)
loan_data_numeric

In [13]:
# the names of the columns 
header_full = np.genfromtxt('loan-data.csv', 
                            delimiter = ';', 
                            autostrip = True, 
                            skip_footer = raw_data_np.shape[0], 
                            dtype = str)
header_full

In [14]:
# to split the headers into two variables, each one holding the name of the column which contain string data or numeric data
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numeric]

In [15]:
header_strings

In [16]:
header_numeric

In [17]:
# creating checkpoint
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + '.npz')
    return(checkpoint_variable)

In [18]:
# testing checkpoint
checkpoint_test = checkpoint('checkpoint-test', header_strings, loan_data_strings)
checkpoint_test['data']

In [19]:
# manipulating string columns
header_strings

In [20]:
header_strings[0] = 'issue_date'

In [21]:
np.unique(loan_data_strings[:,0])

In [22]:
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], '-15')

In [23]:
np.unique(loan_data_strings[:,0])

In [24]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [25]:
for i in range(13):
    loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                     i,
                                     loan_data_strings[:,0])

In [26]:
np.unique(loan_data_strings[:,0])

In [27]:
header_strings

In [28]:
np.unique(loan_data_strings[:,1])

In [29]:
np.unique(loan_data_strings[:,1]).size

In [30]:
status_bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

In [31]:
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], status_bad), 0, 1)

In [32]:
np.unique(loan_data_strings[:,1])

In [33]:
loan_data_strings[:,2]

In [34]:
np.unique(loan_data_strings[:,2])

In [35]:
loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2], ' months')
loan_data_strings[:,2]

In [36]:
header_strings[2] = 'term_months'

In [37]:
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '',
                                     '60',
                                     loan_data_strings[:,2])

In [38]:
np.unique(loan_data_strings[:,2])

In [39]:
np.unique(loan_data_strings[:,3])

In [40]:
np.unique(loan_data_strings[:,4])

In [41]:
# filling sub grade
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                     i + '5',
                                     loan_data_strings[:,4])

In [42]:
np.unique(loan_data_strings[:,4], return_counts = True)

In [43]:
loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '',
                                     'H1',
                                     loan_data_strings[:,4])

In [44]:
np.unique(loan_data_strings[:,4])

In [45]:
# removing grade
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)

In [46]:
loan_data_strings[:,3]

In [47]:
header_strings = np.delete(header_strings, 3)

In [48]:
header_strings

In [49]:
# converting sub grade
keys = list(np.unique(loan_data_strings[:,3]))
# if we don't add +1 at the end, we would only have 35 values for 36 sub_grades
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1))
dict_sub_grade = dict(zip(keys,values))

In [50]:
dict_sub_grade

In [51]:
for i in np.unique(loan_data_strings[:,3]):
    loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i,
                                     dict_sub_grade[i],
                                     loan_data_strings[:,3])

In [52]:
np.unique(loan_data_strings[:,3])

In [53]:
np.unique(loan_data_strings[:,4])

In [54]:
loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') | (loan_data_strings[:,4] == 'Not Verified'), 0, 1)

In [55]:
np.unique(loan_data_strings[:,4])

In [56]:
np.unique(loan_data_strings[:,5])

In [57]:
np.chararray.strip(loan_data_strings[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

In [58]:
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

In [59]:
header_full

In [60]:
loan_data_numeric[:,0].astype(dtype = np.int32)

In [61]:
loan_data_strings[:,5].astype(dtype = np.int32)

In [62]:
np.array_equal(loan_data_numeric[:,0].astype(dtype = np.int32), loan_data_strings[:,5].astype(dtype = np.int32))

In [63]:
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)
header_strings = np.delete(header_strings, 5)

In [64]:
header_strings

In [65]:
header_strings[5] = 'state_address'

In [66]:
np.unique(loan_data_strings[:,5])

In [67]:
np.unique(loan_data_strings[:,5]).size

In [68]:
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]

In [69]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '',
                                     0,
                                     loan_data_strings[:,5])

In [70]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [71]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [72]:
np.unique(loan_data_strings[:,5])

In [73]:
# I've converted whatever data I had into numeric values stored as text
# converting to numbers now
loan_data_strings

In [74]:
loan_data_strings = loan_data_strings.astype(int)

In [75]:
loan_data_strings

In [76]:
# another checkpoint for all the changes that I've made to the former string dataset
checkpoint_strings = checkpoint('Checkpoint-Strings', header_strings, loan_data_strings)
checkpoint_strings['header']

In [77]:
checkpoint_strings['data']

In [78]:
# manipulating numeric columns 
loan_data_numeric

In [79]:
np.isnan(loan_data_numeric).sum()

In [80]:
# substitute "filler" values
temporary_fill

In [81]:
np.isin(loan_data_numeric[:,0], temporary_fill)

In [82]:
np.isin(loan_data_numeric[:,0], temporary_fill).sum()

In [83]:
# 1st row - temporary min
# 2nd row - temporary mean
# 3rd row - temporary max

temporary_stats[:, columns_numeric]

In [84]:
header_numeric

In [85]:
loan_data_numeric[:,2] 

In [86]:
# funded_amnt is the only column for each I need to set the filler values equal to the minimum
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill,
                                     temporary_stats[0, columns_numeric[2]],
                                     loan_data_numeric[:,2])
loan_data_numeric[:,2] 

In [87]:
temporary_stats[0, columns_numeric[3]]

In [88]:
header_numeric

In [89]:
# filling out the remaining numeric columns
for i in [1,3,4,5]:
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temporary_fill,
                                      temporary_stats[2, columns_numeric[i]],
                                      loan_data_numeric[:,i])
    
loan_data_numeric

In [90]:
# currency change
EUR_USD = np.genfromtxt("EUR-USD.csv", delimiter = ',', autostrip = True, skip_header = 1, usecols = 3)
EUR_USD

In [91]:
# creating a new column to store the exchange rates for each account
# I'll start by setting exchange rate equal to the 1st column of the string dataset because it contains all the issue dates

loan_data_strings[:,0]

In [92]:
exchange_rate = loan_data_strings[:,0]

# even though January is represented by 1, the index for EUR_USD must be 0
for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    

exchange_rate = np.where(exchange_rate == 0,
                         np.mean(EUR_USD),
                         exchange_rate)

exchange_rate

In [93]:
exchange_rate.shape

In [94]:
loan_data_numeric.shape

In [95]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [96]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [97]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))

In [98]:
header_numeric

In [99]:
columns_dollar = np.array([1,2,4,5])

In [100]:
loan_data_numeric[:,[columns_dollar]]

In [101]:
loan_data_numeric[:,6]

In [102]:
# the first input will be the entire dataset because I want to attach the new columns to the existing dataset
# then I'll reference a column depending on the iterator variable and divide it by the exchange rate that is stored in the 7th column of the numeric dataset
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

In [103]:
loan_data_numeric.shape

In [104]:
# expanding the header
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])
header_additional

In [105]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [106]:
header_numeric

In [107]:
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]])

In [108]:
header_numeric

In [109]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [110]:
header_numeric = header_numeric[columns_index_order]

In [111]:
header_numeric

In [112]:
loan_data_numeric = loan_data_numeric[:, columns_index_order]

In [113]:
header_numeric

In [114]:
# the convention usually dictates using values between 0 and 1
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100

In [115]:
loan_data_numeric[:,5]

In [116]:
# creating a checkpoint for all the numerical values
checkpoint_numeric = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)

In [117]:
checkpoint_numeric['header'], checkpoint_numeric['data']

In [118]:
# combining the preprocessed versions of the string and numeric values:
checkpoint_strings['data'].shape

In [119]:
checkpoint_numeric['data'].shape

In [120]:
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [121]:
loan_data

In [122]:
np.isnan(loan_data).sum()

In [123]:
# stitching header arrays of the checkpoint_numeric and checkpoint_strings
header_full = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

In [124]:
header_full

In [125]:
# rearranging the entire dataset according to the values in the 1st column 
loan_data = loan_data[np.argsort(loan_data[:,0])]

In [126]:
np.argsort(loan_data[:,0])

In [127]:
# storing the new dataset
loan_data = np.vstack((header_full, loan_data))

In [128]:
np.savetxt("loan-data-preprocessed.csv", 
           loan_data, 
           fmt = '%s',
           delimiter = ',')